# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: Bert-base-uncased
* Evaluation approach: Trainer.evaluate() and compute_metrics
* Fine-tuning dataset: Yelp/yelp_review_full 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, DataCollatorWithPadding, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, TaskType
import numpy as np
import torch

### initialize tokenizer and tokenize_function
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation = True)

In [14]:
### get the dataset for training and evaluating
train_ds, eval_ds = load_dataset("Yelp/yelp_review_full", split=["train[:4000]", "test[:1000]"]) 

# get the labels of the dataset and put them in dictionaries
labels = train_ds.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

# splitting the dataset into train and evaluation and then tokenizing them
train_ds = train_ds.map(tokenize_function, batched=True)
eval_ds = eval_ds.map(tokenize_function, batched=True)

# output data from both training and evaluation data sets
print(train_ds)
eval_ds

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
})


Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
# check to see if data looks right and matches with the dataset page https://huggingface.co/datasets/PolyAI/banking77
print(train_ds[10])
print(eval_ds[10])

In [15]:
### getting the pretrained model
model_name = "bert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 5,
                                                           label2id=label2id, id2label=id2label)

# freeze all parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = False

# creating a compute_metrics function to calculate how accurate the model is during training
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
### training the pre-trained model to the dataset and evaluating its performance

# setting its arguments
training_args = TrainingArguments(output_dir= "./results", 
                                  per_device_train_batch_size= 16, per_device_eval_batch_size= 16, 
                                  learning_rate= 2e-5, num_train_epochs= 0, weight_decay = 0.01,
                                  evaluation_strategy="epoch", save_strategy="epoch",
                                  load_best_model_at_end= True,)

trainer = Trainer(model = model, args = training_args, train_dataset= train_ds, eval_dataset= eval_ds,
                  tokenizer=tokenizer, data_collator= DataCollatorWithPadding(tokenizer=tokenizer), 
                  compute_metrics=compute_metrics,)

# get evaluation stats
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.6740609407424927,
 'eval_accuracy': 0.234,
 'eval_runtime': 33.3252,
 'eval_samples_per_second': 30.007,
 'eval_steps_per_second': 1.89}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
# setting up different LoraConfigs
config = LoraConfig(task_type= TaskType.SEQ_CLS, r = 1,
                    lora_alpha= 1, lora_dropout= 0.01,
                    bias = "none")

# creating the base model before lora and peft configurations
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 5, 
                                             label2id=label2id, id2label=id2label)

# unfreeze all parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = True

In [ ]:
# reset the format of the train and evaluation dataset to contain specific columns
# for the training and evaluating
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

lora_model = get_peft_model(model, config)      # original lora model that will be saved

# see the trainable parameters of each 
lora_model.print_trainable_parameters()

print(lora_model)

In [ ]:
### training the pre-trained model to the dataset and evaluating its performance

# setting its arguments
peft_training_args = TrainingArguments(output_dir= "./peft_test_trainer",
                                  per_device_train_batch_size= 16, per_device_eval_batch_size= 16, 
                                  learning_rate= 2e-5, num_train_epochs= 5, weight_decay= 0.01,
                                  evaluation_strategy= "epoch", save_strategy= "epoch",
                                  load_best_model_at_end= True, logging_dir='./logs',)

peft_trainer = Trainer(model = lora_model, args = peft_training_args, train_dataset= train_ds, eval_dataset= eval_ds,
                       tokenizer = tokenizer, data_collator = DataCollatorWithPadding(tokenizer=tokenizer), 
                       compute_metrics=compute_metrics,)

peft_trainer.train()

lora_model.save_pretrained("peft-bert-lora-model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
peft_trainer.evaluate()

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

# load the base model first
base_model = AutoModelForCausalLM.from_pretrained("bert-base-uncased")

# load the saved peft configurations from the directory we saved above
peft_model_id = "peft-bert-lora-model"

# create model with the peft configurations
new_peft_model = PeftModel.from_pretrained(base_model, peft_model_id)

# keep the peft model with the new weights
new_peft_model.merge_and_unload()

In [ ]:
# evaluating the model
new_peft_model.eval()

### inferencing with the model
inputs = tokenizer("When can I expect my new card to arrive?", return_tensors="pt")

with torch.no_grad():
    outputs = new_peft_model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs))

In [ ]:
### extra lora_configs
config2 = LoraConfig(task_type= TaskType.SEQ_CLS, r = 4, lora_alpha= 16, lora_dropout= 0.2)
config3 = LoraConfig(task_type= TaskType.SEQ_CLS, r = 8, lora_alpha= 32, lora_dropout= 0.1)

lora_model2 = get_peft_model(model, config2)
lora_model3 = get_peft_model(model, config3)

lora_model2.print_trainable_parameters()
lora_model3.print_trainable_parameters()

### training the other two lora models and evaluating their results
trainer2 = Trainer(model = lora_model2, 
                   args = TrainingArguments(output_dir= "./peft_test_trainer2",
                                  per_device_train_batch_size= 16, 
                                  per_device_eval_batch_size= 16, 
                                  learning_rate= 2e-5, 
                                  num_train_epochs= 5, 
                                  weight_decay= 0.01,
                                  evaluation_strategy= "epoch", 
                                  save_strategy= "epoch",
                                  load_best_model_at_end= True,
                                  logging_dir='./logs2',
                    ), 
                   train_dataset= train_ds, 
                   eval_dataset= eval_ds,
                   tokenizer = tokenizer, 
                   data_collator= DataCollatorWithPadding(tokenizer=tokenizer), 
                   compute_metrics=compute_metrics,)
trainer2.train()
trainer2.evaluate()

In [ ]:
trainer3 = Trainer(model = lora_model3, 
                   args = TrainingArguments(output_dir= "./peft_test_trainer3",
                                  per_device_train_batch_size= 16, 
                                  per_device_eval_batch_size= 16, 
                                  learning_rate= 2e-5, 
                                  num_train_epochs= 5, 
                                  weight_decay= 0.01,
                                  evaluation_strategy= "epoch", 
                                  save_strategy= "epoch",
                                  load_best_model_at_end= True,
                                  logging_dir='./logs3',
                    ),  
                   train_dataset= train_ds, 
                   eval_dataset= eval_ds,
                   tokenizer = tokenizer, 
                   data_collator= DataCollatorWithPadding(tokenizer=tokenizer), 
                   compute_metrics=compute_metrics,)
trainer3.train()
trainer3.evaluate()